In [1]:
import os
import configparser
import sqlite3 as sl

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

insert_amd_env_vars =  int(configParser.get('COMMON', 'insert_amd_env_vars'))
HSA_OVERRIDE_GFX_VERSION =  configParser.get('COMMON', 'HSA_OVERRIDE_GFX_VERSION')
ROCM_PATH =  configParser.get('COMMON', 'ROCM_PATH')

if(insert_amd_env_vars != 0):
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = HSA_OVERRIDE_GFX_VERSION
    os.environ["ROCM_PATH"] = ROCM_PATH
datasetPathDatabase =  configParser.get('COMMON', 'datasetPathDatabase') + '/dataset.db'
con = sl.connect(datasetPathDatabase)

In [300]:
import time
start = time.time()
for _ in range(20):
    sql = ("SELECT V.ID,F.ID,V.VIDEO_PATH, V.AGE CAPTION_A, " + 
                        "V.ETHNICITY CAPTION_E, " +
                        "lower(V.GENDER) CAPTION_G, " +
                            "A.SPEAKER_EMB, A.PYANNOTE_TITANET, " +
                        "A.AUDIO_FEATURES, " +
                        "A.LANG CAPTION_L, "+
                        "F.FACE_PATH "+
                        "FROM VIDEO V "+
                        "INNER JOIN AUDIO A ON V.ID = A.VIDEO_ID " +
                        "INNER JOIN FACE F ON F.ID = (select ID from FACE f2 where f2.video_id = v.ID ORDER By ID limit 1 ) " + 
                        "WHERE AUDIO_LENGTH = " + str(24) + ' ' +
                        "AND V.ID in (select V2.ID from VIDEO v2 WHERE V2.TRAINED IS NULL AND AUDIO_PRE IN (3,4) AND FACES_PRE = 2 ORDER BY ABS(RANDOM()) LIMIT "+ str(50) + ")")
    data = con.execute(sql)
    dataGotten = data.fetchall()
end = time.time()
print(end - start)

0.04599404335021973


In [266]:
import time
start = time.time()
for _ in range(1):
    sql = ("SELECT * FROM GENERATOR")
    data = con.execute(sql)
    dataGotten = data.fetchall()
end = time.time()
print(end - start)

0.0037674903869628906


1000 1 times : ~0.07 seconds
500 2 times : ~0.06 seconds
100 10 times: ~0.065 seconds
20 50 TIMES: ~ 0.002 seconds
1 1000 times: ~ 0.002 seconds
VIEW
1000 1 times: ~0.07 seconds
1000 1 times: ~ 0.002 seconds

In [11]:
from numpy import array, nan
import numpy as np
arr = np.array([nan, nan,nan])
n = len(arr)
temp = n*[None]
arr[np.isnan(arr)] = None
temp

[None, None, None]

In [3]:
import pandas as pd
s = pd.Series([12,14,"aaaaa"])
s

0       12
1       14
2    aaaaa
dtype: object

In [ ]:
df_data['gender','ethnicity','age','VGG'] = df_data.apply(extract_face_fn,args=(
    start_at_frame,output_folder,resizeImageTo,fddfb,efvr,efhr,auto_zoom

In [1]:

import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)


datasetPathVideo =  configParser.get('COMMON', 'test_datasetPathVideo')

datasetPathFeatures =  configParser.get('evaluate_imagen', 'test_datasetPathFeatures')
datasetPathGeneratedFaces =  configParser.get('evaluate_imagen', 'test_datasetPathGeneratedFaces')
generated_face_table_name =  configParser.get('evaluate_imagen', 'generated_face_table_name')

datasetPathDatabase =  configParser.get('COMMON', 'test_datasetPathDatabase') + '/dataset.db'

ttwbdf =  int(configParser.get('evaluate_imagen', 'time_to_wait_before_deleting_files'))


cuda =  int(configParser.get('COMMON', 'cuda'))
cpus =  int(configParser.get('COMMON', 'cpus'))

image_size =  int(configParser.get('COMMON', 'resizeImageTo'))
begin_with_image_size = int(configParser.get('COMMON', 'begin_with_image_size'))
timesteps= int(configParser.get('COMMON', 'timesteps') )
unet1_image_size =  int(configParser.get('COMMON', 'unet1_image_size'))
audio_length_used =  configParser.get('evaluate_imagen', 'audio_length_used') 
model_filename =  configParser.get('evaluate_imagen', 'model_filename') + '_'  + audio_length_used +  's.pt'
openl3_mode =  configParser.get('evaluate_imagen', 'openl3_mode')
folder =  configParser.get('evaluate_imagen', 'folder')
audio_embs =  configParser.get('COMMON', 'audio_embs') 
unet1_dim =  int(configParser.get('COMMON', 'unet1_dim'))
unet2_dim =  int(configParser.get('COMMON', 'unet2_dim'))




In [2]:
import torch
from imagen_pytorch import Unet, Imagen, ImagenTrainer,NullUnet
from imagen_pytorch.data import Dataset

unet0 = NullUnet()  # add a placeholder "null" unet for the base unet

unet1 = Unet(
    dim = unet1_dim,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = 3,
    layer_attns = (False, True, True, True),
    layer_cross_attns = (False, True, True, True)
)

unet2 = Unet(
    dim = unet2_dim,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = (2, 4, 8, 8),
    layer_attns = (False, False, False, True),
    layer_cross_attns = (False, False, False, True)
)

#unet = Unet(
#    dim = 32,
#    dim_mults = (1, 2, 4, 8),
#    num_resnet_blocks = 1,
#    layer_attns = (False, False, False, True),
#    layer_cross_attns = False
#)

# imagen, which contains the unet above

#imagen = Imagen(
#    unets = unet,
#    image_sizes = 32,
#    timesteps = 1000
#)

imagen = Imagen(
    unets = (unet0,unet1, unet2),
    image_sizes = (begin_with_image_size,unet1_image_size, image_size),
    timesteps = timesteps,
    cond_drop_prob = 0.1
).cuda()

trainer = ImagenTrainer(
    imagen = imagen,
    split_valid_from_train = True # whether to split the validation dataset from the training
).cuda()



/home/gamal/anaconda3/envs/ds2f_m_i/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-26 23:19:18.037566: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


The base dimension of your u-net should ideally be no smaller than 128, as recommended by a professional DDPM trainer https://nonint.com/2022/05/04/friends-dont-let-friends-train-small-diffusion-models/


In [3]:
import numpy as np
input = torch.from_numpy(np.ones((1,256,768)) * 0.5)
input = input.to(torch.float)
input2 = torch.from_numpy(np.ones((1,3,32,32)) * 0.5)
input2 = input2.to(torch.float)
print(input.shape)
print(input2.shape)

torch.Size([1, 256, 768])
torch.Size([1, 3, 32, 32])


In [4]:
input

tensor([[[0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
         ...,
         [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000]]])

In [4]:
try:
    images = trainer.sample(text_embeds=input,start_image_or_video = input2,
                        stop_at_unet_number=3,start_at_unet_number = 2
                        ,batch_size = 1, return_pil_images = True) # returns List[Image]
except:
    print('s')

: 

In [ ]:
bool("0")

True

In [ ]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import pyarrow as pa

a = np.array([[5, 6], [7, 8]]).flatten()
b = np.array([[9, 10], [11, 12]]).flatten()
df = pd.DataFrame({"col1": [a],"col2":[b]})
df.loc[1] = (b,a)

df = dd.from_pandas(df, npartitions=2)

print(df.compute())

name_function = lambda x: f"data-{x}.parquet"
df[['col1','col2']].to_parquet('train/',name_function=name_function,engine='pyarrow',schema={"col1": pa.list_(pa.int32()),"col2": pa.list_(pa.int32())})


df2 = dd.read_parquet(['train/data-0.parquet','train/data-1.parquet'])  
print(df2.compute())

def reshape(x):
    if(x == 'foo'):
        return x
    return x.reshape(2,2) 

df2 = df2.applymap(lambda x:reshape(x)) 
print(df2.compute())

              col1             col2
0     [5, 6, 7, 8]  [9, 10, 11, 12]
1  [9, 10, 11, 12]     [5, 6, 7, 8]
              col1             col2
0     [5, 6, 7, 8]  [9, 10, 11, 12]
1  [9, 10, 11, 12]     [5, 6, 7, 8]
                  col1                 col2
0     [[5, 6], [7, 8]]  [[9, 10], [11, 12]]
1  [[9, 10], [11, 12]]     [[5, 6], [7, 8]]


/tmp/ipykernel_10218/2638581980.py:18: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(x == 'foo'):
/tmp/ipykernel_10218/2638581980.py:18: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(x == 'foo'):


In [ ]:

df["image_guide"] = np.nan
df.compute()

,col1,col2,col3,image_guide
0,0,1,2,NaN
1,5,6,7,NaN
2,10,11,12,NaN
3,15,16,-1,NaN


In [ ]:
df.set_index("col3")
df.repartition(npartitions=2)
df.compute()

,col1,col2,col3,image_guide
0,0,1,2,NaN
1,5,6,7,NaN
2,10,11,12,NaN
3,15,16,-1,NaN


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df.compute()

,col1,col2,image_guide
0,1,3,NaN
0,2,4,NaN


In [ ]:
data_frame = df[["col1","col2"]]
data_frame.compute()

,col1,col2
0,1,3
0,2,4


In [ ]:
meta_df = pd.DataFrame(columns=["x", "y","z"], dtype=int)
meta_df

,x,y,z


In [ ]:
def speaker_emb_preprocess(x):
    return x*x*x

data_frame['col3'] = data_frame['col1'].apply(lambda x: speaker_emb_preprocess(x),meta=('1', 'object'))
data_frame

,col1,col2,col3
npartitions=2,,,
,int64,int64,object
,...,...,...
,...,...,...


In [ ]:
meta_df = pd.DataFrame([[1,2,4],[1,4,5]], dtype=int)
type("meta_df")

str

In [ ]:
type(np.zeros((1,768)).tolist())

list

In [ ]:
data_frame = data_frame.drop(['col1'], axis=1)
data_frame

,col2,col3
npartitions=2,,
,int64,object
,...,...
,...,...


In [ ]:
data_frame[['col2','col3']]

,col2,col3
npartitions=2,,
,int64,object
,...,...
,...,...


In [ ]:
np.array(data_frame["col2"])

array([3, 4])

In [ ]:
for index, row in data_frame.iterrows():
    print(row['col2'])

3
4


In [ ]:
data_frame.loc[1,"col2"]=2

TypeError: '_LocIndexer' object does not support item assignment

In [ ]:
from matplotlib import pyplot as plt
x = []
x.append(0)
x.append(1)
x.append(2)
x

In [ ]:
fig = plt.figure()
plt.plot(range(10))

plt.title("Training Loss")
plt.xlabel("Training Sample (x500)")
plt.ylabel("MSE Loss")
fig.savefig("save_file_name.png")
plt.close()

In [ ]:
import random
# For a single Element
random.choices([True, False], weights=[100, 0])[0]

In [ ]:
from PIL import Image
import numpy as np


image_size = 128

def make_square(im, min_size=128, fill_color=(0, 0, 0, 0)):
    x, y = im.size
    size = max(min_size, x, y)
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    im.close()
    return new_im

def image_guide_preprocess_low_res_dummy(path):
    #print(face_id)

    image_guide = np.zeros((1, 768))
    return image_guide

def image_guide_preprocess_low_res(path):
    #print(face_id)


    if(False): #never
        image_guide = np.zeros((49, 768))
        #print(image_guide.shape)
    else:

        image = Image.open(path).convert('RGB')

        #print(image.size)

        w_s = image_size / (1+2 * 0.4)
        h_s = image_size / (1+2 * 0.4)

        image = image.crop((0.2*w_s, 0.0*h_s, 1.6*w_s, 1.4*h_s))
        image = image.resize((128,128))
        print(image.size)
        #image = im

        #image = image.resize((16,16))
        im = image
        

        print('saving')
        image.save('opop.png')

        #print(np.array(image,np.float32).shape)

        pix = np.array(image, np.float32)
        pix = np.moveaxis(pix, -1, 0)

        pix = pix / 255
        image.close()
        im.close()
        return pix.tolist()
    return image_guide


In [ ]:
im = '/media/gamal/Passport/Datasets/VoxCeleb2Test/Voxceleb2TestFaces/mp4/id01106/45TAeIEqhcU/00001_face_1.png'

In [ ]:
print(im)

In [ ]:
image_guide_preprocess_low_res(im)

In [ ]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [ ]:
from itertools import islice
import csv
import sys
import os
import glob
csv.field_size_limit(sys.maxsize)
class CsvDataset(Dataset):

    def __init__(self, csv_folder):

        self.csv_folder = csv_folder

    def __len__(self):
        _, _, files = next(os.walk(self.csv_folder))
        file_count = len(files)
        print('found ' + str(file_count) + ' files.')
        return file_count

    def __getitem__(self, index):
        file = sorted(glob.glob(self.csv_folder + '*.csv'))[index]
        df = pd.read_csv(file)
        print(df.head(1))
        df['INPUT'] = np.fromstring(s[1:-1], dtype=np.int, sep=' ')
        #df['INPUT'] = df['INPUT'].astype('numpy.ndarray')
        print(df['INPUT'][0])
        torch_tensor = torch.tensor(df.head(1).values)
        return torch_tensor

In [ ]:
face_dataset = CsvDataset(csv_folder='train/')

In [ ]:
train_loader = torch.utils.data.DataLoader(face_dataset,
        batch_size=1)

In [ ]:
for x in train_loader:
    print(x)
    break

found 1 files.
   Unnamed: 0                                              INPUT  \
0           0  [[0. 0. 0. ... 0. 0. 1.]\n [0. 0. 0. ... 0. 0....   

                                          image_path  \
0  [[[0.23921569 0.23921569 0.23921569 ... 0.3019...   

                                       low_res_image  
0  [[[0.24313726 0.6313726  0.6784314  0.7529412 ...  


TypeError: data type 'numpy.ndarray' not understood

In [ ]:
names = np.arange(10)
charar = np.chararray((1, 10))
charar[:] = 'i'
names = charar.astype('str') + names.astype('str')
meta_df = pd.DataFrame(columns=names.astype(str), dtype=int)
meta_df

,"(i0, i1, i2, i3, i4, i5, i6, i7, i8, i9)"


In [261]:


import numpy as np
from sklearn.metrics import mean_squared_error

# Set seed for reproducibility
#np.random.seed(42)

# Generate two vectors of length 2622 with values ranging from -1 to 1
vector1 = np.random.uniform(-1, 1, 2622)
vector2 = np.ones(2622)


# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(vector1, vector2)

print(f"Vector 1: {vector1}")
print(f"Vector 2: {vector2}")
print(f"Mean Squared Error: {mse}")

Vector 1: [-0.86517422  0.11121536 -0.87335438 ... -0.3587972   0.8138974
 -0.51492585]
Vector 2: [1. 1. 1. ... 1. 1. 1.]
Mean Squared Error: 1.3632427047174027


In [262]:
import pickle
import zlib
import sys
print(sys.getsizeof(pickle.dumps(vector2)))
print(sys.getsizeof(zlib.compress(vector1,level=0)))
print(sys.getsizeof(zlib.compress(vector1,level=9)))
print(vector1)
x = zlib.compress(vector1)
y = zlib.decompress(x)
y

21160
21020
20077
[-0.86517422  0.11121536 -0.87335438 ... -0.3587972   0.8138974
 -0.51492585]


b'\xaa\x0e\xfa\xd9\x81\xaf\xeb\xbf0{\x83 \x9cx\xbc?P\x81\x0b\xe3\x84\xf2\xeb\xbf\xfc\x8c\xfb&\xd4\xd1\xe6?x\x810\x14\xe6\x84\xd5\xbf\xa4\xf3\xe7r\xd4g\xe9\xbf\xd8\xa0r\xad\x84\xa2\xc5?P\xfc\xcd\x1a\xad\x0b\xb8?\xf4##\x18\xce2\xd1?\xc2\xd8\xd5\x16\xbd\xa0\xe7\xbf\x1c0\xc8\x7f\x02\x19\xe9?\xd0"+\xa4(\xf0\xb4\xbf\x88\x11%%\x01\xad\xc2?\\\xc3\xdb9\xd23\xe1?\xa8\x97\x9d\xfd$2\xe8\xbf \xa1\xa0ib\x03\xd4?8\xbax\x88kv\xd1?\xf0\xdd\n\n\xcc\x9b\xd4\xbf8\xe0\xb7XT\xaa\xc7?\xf4(\xe15\t\x85\xd8?\xd8\xa6F\xb3x\x03\xcd?\xf0\x90\xbaO\x8bi\xe5\xbf\xcc\xcd\x81\xf7Gn\xeb?\xc4\xda\xddM\xfd\x89\xe4\xbf\xc0W\xeb9\xf6\x87\xd1\xbfD\xd2\xd8\x02\xa4A\xeb\xbf\x94|\xea\xbd\xcc\xd2\xdf?\xc0:!kb\xad\xe3\xbf\xaa\xb0\x07`g\xc7\xe2?\x98{\xcc^\x0fk\xe7?\xd2\xc6t\xe2\x0bZ\xe5\xbf(\x89s\x7f67\xd0?\xac\xacA\x86\xed\t\xef\xbf\xdc\xe740l\x01\xd2?\xd6#/\xa1\x82\x83\xe3?p\xb9:e|\xd7\xd6\xbfh\xd2\xf2\xe0*\x02\xd8\xbf\xe6.\x8f\x18\\\xb6\xe8?\xda\x88\xe9\xa5z\xb0\xed?\xbc\x8d\xfaU\xac\xcb\xec?\xf4\xe2\xc2EQW\xd3?N\xa3\x15\xec\xa

In [207]:
import bz2
import gzip
import lzma
import pickle



class SomeObject():

    a = 'some data'
    b = 123
    c = 'more data'

    def __init__(self, i):
        self.i = i


data = vector1

with open('no_compression.pickle', 'wb') as f:
    pickle.dump(data, f, protocol=5)

with gzip.open("gzip_test.gz", "wb", compresslevel=9) as f:
    pickle.dump(data, f)

with bz2.BZ2File('bz2_test.pbz2', 'wb') as f:
    pickle.dump(data, f)

with lzma.open("lzma_test.xz", "wb") as f:
    pickle.dump(data, f)


